In [1]:
import numpy as np
import tensorflow as tf

# 设置按需使用GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

import time

/Users/stephaniexia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 用tensorflow 导入数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=False)

print(mnist.test.labels.shape)
print(mnist.train.labels.shape)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
(10000,)
(55000,)


In [14]:
with tf.name_scope('inputs'):
    X_ = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder(tf.int64, [None])

# 把X转为卷积所需要的形式
with tf.name_scope('conv_part'):
    X = tf.reshape(X_, [-1, 28, 28, 1])
    h_conv1 = tf.layers.conv2d(X, filters=32, kernel_size=5, strides=1, padding='same', activation=tf.nn.relu, name='conv1',reuse=tf.AUTO_REUSE)
    h_pool1 = tf.layers.max_pooling2d(h_conv1, pool_size=2, strides=2, padding='same', name='pool1')

    h_conv2 = tf.layers.conv2d(h_pool1, filters=64, kernel_size=5, strides=1, padding='same',activation=tf.nn.relu, name='conv2',reuse=tf.AUTO_REUSE)
    h_pool2 = tf.layers.max_pooling2d(h_conv2, pool_size=2, strides=2, padding='same', name='pool2')
    print(X,'\n',h_conv1,'\n',h_pool1,'\n',h_conv2,'\n',h_pool2)
    # flatten
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.layers.dense(h_pool2_flat, 1024, name='fc1', activation=tf.nn.relu,reuse=tf.AUTO_REUSE)

    # dropout: 输出的维度和h_fc1一样
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, 0.7)
    print(h_fc1_drop)

CNN_stack=tf.placeholder(tf.float32, [1,10,1024])
with tf.name_scope('lstm_part'):
    lstm_cell_1 = tf.contrib.rnn.LayerNormBasicLSTMCell(512,reuse=tf.AUTO_REUSE)
    print(lstm_cell_1)
    lstm_cell_2 = tf.contrib.rnn.LayerNormBasicLSTMCell(256,reuse=tf.AUTO_REUSE)
    print(lstm_cell_2)
    mul_lstm_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell_1,lstm_cell_2])
    print(mul_lstm_cell)
    init_state = mul_lstm_cell.zero_state(1, dtype=tf.float32)
    outputs, state = tf.nn.dynamic_rnn(mul_lstm_cell, inputs=CNN_stack, initial_state=init_state, time_major=False)
    h_out = outputs
    h_fc2 = tf.layers.dense(outputs, units=10, name='fc2',reuse=tf.AUTO_REUSE)
# y_conv = tf.nn.softmax(h_fc2)
y_conv = tf.reshape(h_fc2,(10,10))
print(y_conv)
#print('Finished building network.')

Tensor("conv_part_8/Reshape:0", shape=(?, 28, 28, 1), dtype=float32) 
 Tensor("conv_part_8/conv1/Relu:0", shape=(?, 28, 28, 32), dtype=float32) 
 Tensor("conv_part_8/pool1/MaxPool:0", shape=(?, 14, 14, 32), dtype=float32) 
 Tensor("conv_part_8/conv2/Relu:0", shape=(?, 14, 14, 64), dtype=float32) 
 Tensor("conv_part_8/pool2/MaxPool:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("conv_part_8/dropout/mul:0", shape=(?, 1024), dtype=float32)
Tensor("Reshape:0", shape=(10, 10), dtype=float32)


In [54]:
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=tf.cast(y_, dtype=tf.int32), logits=y_conv))
train = tf.train.AdamOptimizer(1e-4)
gradient_all = train.compute_gradients(cross_entropy)

sess.run(tf.global_variables_initializer())
saver=tf.train.Saver(max_to_keep=3)
for i in range(5):#epochs
    _h_fc1_drop_stack=[]
    label=np.array([])
    for j in range(5):
        batch = mnist.train.next_batch(2)
        _h_fc1_drop=sess.run(h_fc1_drop,feed_dict={X_:batch[0]})
        _h_fc1_drop_stack.append(_h_fc1_drop)
        label=np.concatenate([label,batch[1]])
    h_fc1_drop_stack=sess.run(tf.reshape(tf.concat(_h_fc1_drop_stack,axis=0),(1,10,1024)))
    #label=sess.run(tf.reshape(tf.concat(label,axis=1),(10)))
    print(label)
    #print(h_fc1_drop_stack)
    #loss,grad,_=sess.run([cross_entropy,gradient,train],feed_dict={CNN_stack:h_fc1_drop_stack,y_:label})
    #loss,_=sess.run([cross_entropy,train],feed_dict={CNN_stack:h_fc1_drop_stack,y_:label})
    #saver.save(sess,'/Users/stephaniexia/Documents/AI tutorial/tensorflow tutorial/mnistCNN_3.ckpt',global_step=i+1)
    #gradient_all = train.compute_gradients(cross_entropy)
    grad=sess.run(gradient_all,feed_dict={CNN_stack:h_fc1_drop_stack,y_:label})
    print(grad)
    
        

[7. 6. 3. 0. 5. 0. 2. 3. 6. 3.]


TypeError: Fetch argument None has invalid type <class 'NoneType'>

In [50]:

from tensorflow.python import pywrap_tensorflow
checkpoint_path =' /Users/stephaniexia/Documents/AI tutorial/tensorflow tutorial/checkpoint'
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map = reader.get_variable_to_shape_map()
for key in var_to_shape_map:
    print("tensor_name: ", key)
    print(reader.get_tensor(key))

InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on  /Users/stephaniexia/Documents/AI tutorial/tensorflow tutorial/checkpoint: Not found:  /Users/stephaniexia/Documents/AI tutorial/tensorflow tutorial; No such file or directory

In [4]:
print(h_conv1)
print(h_pool1)
print(h_conv2)
print(h_pool2)

print(h_pool2_flat)
print(h_fc1)
print(h_fc2)

Tensor("conv1/Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("pool1/MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("conv2/Relu:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("pool2/MaxPool:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
Tensor("fc1/Relu:0", shape=(?, 1024), dtype=float32)
Tensor("fc2/BiasAdd:0", shape=(?, 10), dtype=float32)


In [5]:
# cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=tf.cast(y_, dtype=tf.int32), logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), y_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess.run(tf.global_variables_initializer())

is_train=False
saver=tf.train.Saver(max_to_keep=3)

if is_train:
    max_acc=0
    f=open('/Users/stephaniexia/Documents/AI tutorial/tensorflow tutorial/acc.txt','w')
    tic = time.time()
    for i in range(1000):
        batch = mnist.train.next_batch(100)
        if i%100 == 0:
            #print(len(batch),batch[0].shape)
            train_accuracy = accuracy.eval(feed_dict={
                X_:batch[0], y_: batch[1], keep_prob: 1.0})
            print("step {}, training accuracy = {:.4f}, pass {:.2f}s ".format(i, train_accuracy, time.time() - tic))
        train_step.run(feed_dict={X_: batch[0], y_: batch[1]})
        f.write(str(i+1)+', train_accuracy: '+str(train_accuracy)+'\n')
        if train_accuracy>max_acc:
          max_acc=train_accuracy
          saver.save(sess,'/Users/stephaniexia/Documents/AI tutorial/tensorflow tutorial/mnistCNN_3.ckpt',global_step=i+1)
    f.close()
else:
    model_file=tf.train.latest_checkpoint('/Users/stephaniexia/Documents/AI tutorial/tensorflow tutorial/')
    saver.restore(sess,model_file)
    print("test accuracy %g"%accuracy.eval(feed_dict={
        X_: mnist.test.images, y_: mnist.test.labels}))

step 0, training accuracy = 0.1000, pass 0.24s 
step 100, training accuracy = 0.8700, pass 20.54s 
step 200, training accuracy = 0.9400, pass 40.60s 
step 300, training accuracy = 0.9300, pass 59.70s 
step 400, training accuracy = 0.9900, pass 79.72s 
step 500, training accuracy = 0.9400, pass 99.54s 
step 600, training accuracy = 0.9800, pass 119.51s 
step 700, training accuracy = 0.9700, pass 138.89s 
step 800, training accuracy = 0.9700, pass 159.10s 
step 900, training accuracy = 0.9800, pass 179.25s 
test accuracy 0.9757
